### segment and vect benchmark for each frame of different model (hdmapnet + lss)

In [1]:
from PIL import Image, ImageFile
import matplotlib.pyplot as plt
import cv2
import os
import re

def extract_timestamp_from_filename(filename, keyword):
    """从文件名中提取时间戳"""
    if keyword in filename:
        match = re.search(rf"{re.escape(keyword)}_(\d+\.\d+)\.jpg", filename)
        if match:
            return float(match.group(1))
    return None

def get_subfolders_with_timestamps(path, keyword="2_pvimg_before_norm_CAM_FRONT"):
    """获取路径下所有子文件夹及其对应的时间戳"""
    subfolders = {}
    for subfolder in os.listdir(path):
        subfolder_path = os.path.join(path, subfolder)
        if os.path.isdir(subfolder_path):
            for file in os.listdir(subfolder_path):
                if file.endswith(".jpg"):
                    timestamp = extract_timestamp_from_filename(file, keyword)
                    if timestamp is not None:
                        subfolders[timestamp] = subfolder
                        break
    return subfolders

def match_subfolders_by_timestamp(path1, path2):
    """匹配两个路径下的子文件夹，根据时间戳生成输出列表"""
    path1_subfolders = get_subfolders_with_timestamps(path1)
    path2_subfolders = get_subfolders_with_timestamps(path2)

    result = []
    for timestamp, subfolder1 in path1_subfolders.items():
        subfolder2 = path2_subfolders.get(timestamp, None)
        result.append([timestamp, subfolder1, subfolder2])

    return sorted(result, key=lambda x: x[0])

def find_image_by_keyword(folder, keyword, exclude_keyword=None):
    """在指定文件夹中查找包含特定关键词且不包含排除关键词的图像文件"""
    for file in os.listdir(folder):
        if keyword in file and file.endswith(".jpg"):
            if exclude_keyword and exclude_keyword in file:
                continue
            return os.path.join(folder, file)
    return None

def create_combined_image(images_to_combine, output_path):
    """创建整合图像并保存"""
    widths = [sum(img.width for img in row) for row in images_to_combine]
    heights = [max(img.height for img in row) for row in images_to_combine]

    total_width = max(widths)
    total_height = sum(heights)

    combined_image = Image.new("RGB", (total_width, total_height), (255, 255, 255))  # 创建白色背景

    y_offset = 0
    for row in images_to_combine:
        x_offset = 0
        for img in row:
            combined_image.paste(img, (x_offset, y_offset))
            x_offset += img.width
        y_offset += max(img.height for img in row)

    combined_image.save(output_path)

def resize_image_to_match_height(image, target_height):
    """调整图像高度以匹配目标高度，并保持宽高比"""
    width, height = image.size
    scale_factor = target_height / height
    new_size = (int(width * scale_factor), target_height)
    return image.resize(new_size, Image.Resampling.LANCZOS)

def process_and_save_images(output_folder, path1, path2, result):
    """处理并保存整合图像"""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for timestamp, subfolder1, subfolder2 in result:
        images_to_combine = []

        # 第一行图像
        row1 = []
        for keyword in ["1_pvimg_before_norm_CAM_FRONT", "2_pvimg_before_norm_CAM_FRONT", "3_pvimg_before_norm_CAM_FRONT_RIGHT"]:
            img_path = find_image_by_keyword(os.path.join(path1, subfolder1), keyword)
            if img_path:
                row1.append(Image.open(img_path))
        images_to_combine.append(row1)

        # 第二行图像
        row2 = []
        for keyword in ["6_pvimg_before_norm_CAM_BACK_RIGHT", "5_pvimg_before_norm_CAM_BACK", "4_pvimg_before_norm_CAM_BACK_LEFT"]:
            img_path = find_image_by_keyword(os.path.join(path1, subfolder1), keyword)
            if img_path:
                row2.append(Image.open(img_path))
        images_to_combine.append(row2)

        # 第三行图像
        row3 = []
        img_path1 = find_image_by_keyword(os.path.join(path1, subfolder1), "segmentation")
        if img_path1:
            img1 = Image.open(img_path1)
            row3.append(img1)
        img_path2 = find_image_by_keyword(os.path.join(path2, subfolder2), "segment_LSS_all")
        if img_path2:
            img2 = Image.open(img_path2)
            img2_resized = resize_image_to_match_height(img2, img1.height)  # 调整高度
            row3.append(img2_resized)
        images_to_combine.append(row3)

        # 第四行图像
        row4 = []
        img_path1 = find_image_by_keyword(os.path.join(path1, subfolder1), "eval_vector", exclude_keyword="cluster")
        if img_path1:
            img1 = Image.open(img_path1)
            row4.append(img1)
        img_path2 = find_image_by_keyword(os.path.join(path2, subfolder2), "eval_vector", exclude_keyword="cluster")
        if img_path2:
            img2 = Image.open(img_path2)
            img2_resized = resize_image_to_match_height(img2, img1.height)  # 调整高度
            row4.append(img2_resized)
        images_to_combine.append(row4)

        # 创建整合图像并保存
        output_path = os.path.join(output_folder, f"{timestamp}.jpg")
        create_combined_image(images_to_combine, output_path)

# 示例路径
PATH1 = "/home/sherlock/Documents/Projects/HDMapNet-test/plt_images/MB_dataset2/n005_E33_wb"
PATH2 = "/home/sherlock/Pictures/hmn_experients_output/HDMapNet/02.08_LSS/2024_8_2_15_49_44"
OUTPUT_FOLDER = "./benchmarks"

output = match_subfolders_by_timestamp(PATH1, PATH2)
process_and_save_images(OUTPUT_FOLDER, PATH1, PATH2, output)
# for row in output:
#     print(row)




### histogram generator (not necessary)

In [1]:
from PIL import Image, ImageFile
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

MATERIAL_PATH = './material_images'
INTEGRATE_PATH = './integrate_histograms'

# 确保输出目录存在
if not os.path.exists(INTEGRATE_PATH):
    os.makedirs(INTEGRATE_PATH)

# 允许加载截断的图片文件
ImageFile.LOAD_TRUNCATED_IMAGES = True

def get_subfolders(directory):
    """
    一个生成器函数，遍历指定目录下的所有子文件夹。
    
    Args:
        directory (str): 需要遍历的目录路径。
    
    Yields:
        str: 子文件夹的完整路径。
    """
    for root, dirs, files in os.walk(directory):
        # 仅在顶层目录中查找子目录
        for name in dirs:
            yield os.path.join(root, name)
        break  # 防止深入子文件夹

def read_image(path):
    """
    读取图像文件
    
    Args:
        path (str): 图像文件的路径。
    
    Returns:
        Image: 读取的图像对象。
    """
    return Image.open(path)

def plot_histogram(image):
    """
    绘制图像的RGB直方图
    
    Args:
        image (Image): 图像对象。
    
    Returns:
        Image: 绘制直方图的图像对象。
    """
    fig, ax = plt.subplots(1, 1, figsize=(6, 3))
    hist_max = 0  # 用于存储最大直方图值

    for i, color in enumerate(['r', 'g', 'b']):
        hist = cv2.calcHist([np.array(image)], [i], None, [256], [0, 256])
        hist_max = max(hist_max, hist.max())  # 更新最大直方图值
        ax.plot(hist / hist_max, color=color)  # 归一化直方图值并绘图

    ax.set_xlim([0, 256])
    ax.set_ylim([0, 1])
    ax.set_title('Histogram')

    # 将绘图对象转换为图像对象
    fig.canvas.draw()
    hist_img = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    hist_img = hist_img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)
    return Image.fromarray(hist_img)

def histogram_displayer(MATERIAL_PATH, INTEGRATE_PATH):
    # 获取所有子文件夹
    subfolders = list(get_subfolders(MATERIAL_PATH))

    for subfolder in subfolders:
        pvimg_before_norm = [os.path.join(subfolder, f) for f in os.listdir(subfolder) 
                             if '2_pvimg_before_norm' in f and f.endswith('.jpg')]
        fm_imgafter_norm_ImageNet = [os.path.join(subfolder, f) for f in os.listdir(subfolder) 
                                     if '2_fm_imgafter_norm' in f and 'ImageNet' in f and f.endswith('.jpg')]
        fm_imgafter_norm_MB = [os.path.join(subfolder, f) for f in os.listdir(subfolder) 
                               if '2_fm_imgafter_norm' in f and 'MB' in f and f.endswith('.jpg')]

        print(f"Found images: {pvimg_before_norm}, {fm_imgafter_norm_ImageNet}, {fm_imgafter_norm_MB}")

        if not (pvimg_before_norm and fm_imgafter_norm_ImageNet and fm_imgafter_norm_MB):
            print(f"Missing images in subfolder: {subfolder}")
            continue

        images = [read_image(pvimg_before_norm[0]), read_image(fm_imgafter_norm_ImageNet[0]), read_image(fm_imgafter_norm_MB[0])]

        if any(img is None for img in images):
            print(f"Error loading images in subfolder: {subfolder}")
            continue

        # 创建一个白色的画布
        canvas_width = 1920
        canvas_height = 1080
        canvas = Image.new('RGB', (canvas_width, canvas_height), (255, 255, 255))

        # 粘贴三张图片到画布上
        img_width, img_height = images[0].size
        for i, img in enumerate(images):
            canvas.paste(img, (i * img_width, 0))

        # 绘制直方图并粘贴到画布上
        hist_height = 360
        for i, img in enumerate(images):
            hist_img = plot_histogram(img)
            hist_img = hist_img.resize((img_width, hist_height))
            canvas.paste(hist_img, (i * img_width, img_height))

        # 保存最终的集成图像
        final_img_path = os.path.join(INTEGRATE_PATH, os.path.basename(subfolder) + '_integrated.png')
        canvas.save(final_img_path)
        print(f"Saved integrated image to {final_img_path}")

if __name__ == '__main__':
    histogram_displayer(MATERIAL_PATH, INTEGRATE_PATH)


Found images: ['./material_images/kLcf5dyqVBQZ2OrPdecBgy5Z49PptQTq/2_pvimg_before_norm_CAM_FRONT_5424.4.jpg'], [], []
Missing images in subfolder: ./material_images/kLcf5dyqVBQZ2OrPdecBgy5Z49PptQTq
Found images: ['./material_images/F6leDVdKDv0at4tFmuMELVwobB9T99gG/2_pvimg_before_norm_CAM_FRONT_5426.4.jpg'], [], []
Missing images in subfolder: ./material_images/F6leDVdKDv0at4tFmuMELVwobB9T99gG
Found images: ['./material_images/7kcxdkAXQbriTPWWzZlEactu6rlcGIxX/2_pvimg_before_norm_CAM_FRONT_5421.4.jpg'], [], []
Missing images in subfolder: ./material_images/7kcxdkAXQbriTPWWzZlEactu6rlcGIxX
Found images: ['./material_images/o5f6trtJm7HSTBMTJepJ4u1W7fUIlxVE/2_pvimg_before_norm_CAM_FRONT_5423.4.jpg'], [], []
Missing images in subfolder: ./material_images/o5f6trtJm7HSTBMTJepJ4u1W7fUIlxVE
Found images: ['./material_images/jyGG9MMYqfRoU2wGKoJuNnSn11gTeOW1/2_pvimg_before_norm_CAM_FRONT_5423.9.jpg'], [], []
Missing images in subfolder: ./material_images/jyGG9MMYqfRoU2wGKoJuNnSn11gTeOW1
Found imag

### 6 view camera images (6 before norm and 6+6 after norm) + 2 segmentation images

In [5]:
from PIL import Image, ImageFile
import os
import re
import cv2
import numpy as np

MATERIAL_PATH = './material_images'
INTEGRATE_PATH = './integrate_images'

# 确保输出目录存在
if not os.path.exists(INTEGRATE_PATH):
    os.makedirs(INTEGRATE_PATH)

# 允许加载截断的图片文件
ImageFile.LOAD_TRUNCATED_IMAGES = True

def get_subfolders(directory):
    """
    一个生成器函数，遍历指定目录下的所有子文件夹。
    
    Args:
        directory (str): 需要遍历的目录路径。
    
    Yields:
        str: 子文件夹的完整路径。
    """
    for root, dirs, files in os.walk(directory):
        # 仅在顶层目录中查找子目录
        for name in dirs:
            yield os.path.join(root, name)
        break  # 防止深入子文件夹

def normalize_img_displayer(MATERIAL_PATH, INTEGRATE_PATH):
    # 获取所有子文件夹
    subfolders = list(get_subfolders(MATERIAL_PATH))

    for subfolder in subfolders:
        # 获取符合条件的图像文件
        pvimg_before_norm = sorted([os.path.join(subfolder, f) for f in os.listdir(subfolder) if 'pvimg_before_norm' in f and f.endswith('.jpg')])
        fm_imgafter_norm_ImageNet = sorted([os.path.join(subfolder, f) for f in os.listdir(subfolder) if 'fm_imgafter_norm' in f and 'brightUp' in f and f.endswith('.jpg')])
        fm_imgafter_norm_MB = sorted([os.path.join(subfolder, f) for f in os.listdir(subfolder) if 'fm_imgafter_norm' in f and 'org' in f and f.endswith('.jpg')])
        segmentation_imgs = sorted([os.path.join(subfolder, f) for f in os.listdir(subfolder) if 'segmentation' in f and f.endswith('.jpg')])

        # 创建一个 1920x1080 的空白图像
        canvas = Image.new('RGB', (1920, 1080), (255, 255, 255))

        # 定义粘贴位置的偏移量
        positions = [
            (0, 0), (640, 0), (1280, 0),
            (0, 360), (640, 360), (1280, 360),
            (0, 720), (640, 720), (1280, 720),
            (0, 1080), (640, 1080), (1280, 1080),
            (0, 1440), (640, 1440), (1280, 1440),
            (0, 1800), (640, 1800), (1280, 1800)
        ]

        all_images = pvimg_before_norm + fm_imgafter_norm_ImageNet + fm_imgafter_norm_MB

        # 粘贴所有图像
        for i, img_path in enumerate(all_images[:18]):
            img = Image.open(img_path)
            img = img.resize((img.width , img.height))  # 缩放图像
            x_offset = (i % 3) * img.width
            y_offset = (i // 3) * img.height
            canvas.paste(img, (x_offset, y_offset))

        # 粘贴 segmentation 图像
        seg_y = 0
        for img_path in segmentation_imgs:
            img = Image.open(img_path)
            img_width, img_height = img.size
            x_position = 1920 - img_width
            y_position = seg_y
            canvas.paste(img, (x_position, y_position))
            seg_y += img_height + 5  # 每张图像之间间隔5个像素

        # 保存合并后的图像
        if segmentation_imgs:
            base_name = re.sub(r'_\w+$', '', os.path.basename(segmentation_imgs[0]))
            save_path = os.path.join(INTEGRATE_PATH, f'{base_name}.jpg')
            canvas.save(save_path)
            print(f"Saved integrated image to {save_path}")

if __name__ == '__main__':
    normalize_img_displayer(MATERIAL_PATH, INTEGRATE_PATH)

Saved integrated image to ./integrate_images/segmentation_000032_kLcf5dyqVBQZ2OrPdecBgy5Z49PptQTq_000_brightUp.jpg.jpg
Saved integrated image to ./integrate_images/segmentation_000036_F6leDVdKDv0at4tFmuMELVwobB9T99gG_000_brightUp.jpg.jpg
Saved integrated image to ./integrate_images/segmentation_000026_7kcxdkAXQbriTPWWzZlEactu6rlcGIxX_000_brightUp.jpg.jpg
Saved integrated image to ./integrate_images/segmentation_000030_o5f6trtJm7HSTBMTJepJ4u1W7fUIlxVE_000_brightUp.jpg.jpg
Saved integrated image to ./integrate_images/segmentation_000031_jyGG9MMYqfRoU2wGKoJuNnSn11gTeOW1_000_brightUp.jpg.jpg
Saved integrated image to ./integrate_images/segmentation_000038_Ol1zuL3Qxa6qR0HNhdczcussOeXI4jQM_000_brightUp.jpg.jpg
Saved integrated image to ./integrate_images/segmentation_000012_19MBkOV1BAMtH28loWOpOehXU0PneLrL_000_brightUp.jpg.jpg
Saved integrated image to ./integrate_images/segmentation_000018_19uyTNWYMa4aPuoZL0O1sog1L41ZKmIk_000_brightUp.jpg.jpg
Saved integrated image to ./integrate_images/seg

### 2 segmentation images + 6 view camera images

In [3]:
"""
Script program
Author@Shiqi Jiang, xx.04.2024
"""

from PIL import Image, ImageFile
import os
import re
import numpy as np

MATERIAL_PATH = './material_images'
INTEGRATE_PATH = './integrate_images'
MAPTR_OUTPUT_PATH = '/home/sherlock/Pictures/mtr_outputs'

# 确保输出目录存在
if not os.path.exists(INTEGRATE_PATH):
    os.makedirs(INTEGRATE_PATH)

# 允许加载截断的图片文件
ImageFile.LOAD_TRUNCATED_IMAGES = True

def get_subfolders(directory):
    """
    一个生成器函数，遍历指定目录下的所有子文件夹。
    
    Args:
        directory (str): 需要遍历的目录路径。
    
    Yields:
        str: 子文件夹的完整路径。
    """
    for root, dirs, files in os.walk(directory):
        # 仅在顶层目录中查找子目录
        for name in dirs:
            yield os.path.join(root, name)
        break  # 防止深入子文件夹

def extract_min_number_from_folder(folder_path):
    """
    遍历指定文件夹中的所有图像文件，提取每个文件名中的六位整数，并返回最小的整数。
    
    Args:
        folder_path (str): 文件夹路径。
    
    Returns:
        int: 文件夹中所有图像文件名中的最小六位整数，如果没有找到任何整数，则返回None。
    """
    min_number = None
    for filename in os.listdir(folder_path):
        if filename.startswith('segmentation_') and filename.endswith('.jpg'):
            match = re.search(r'segmentation_(\d{6})_', filename)
            if match:
                number = int(match.group(1))
                if min_number is None or number < min_number:
                    min_number = number
    return min_number

def sort_folders_by_min_image_number(base_path):
    """
    根据子文件夹中图像文件名中的最小六位整数对子文件夹进行排序。
    
    Args:
        base_path (str): 总文件夹的路径。
    
    Returns:
        list: 排序后的子文件夹路径列表。
    """
    folders_with_numbers = []
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            min_number = extract_min_number_from_folder(folder_path)
            if min_number is not None:
                folders_with_numbers.append((folder_path, min_number))

    # 根据每个文件夹中图像文件名的最小整数进行排序
    sorted_folders = [folder for folder, _ in sorted(folders_with_numbers, key=lambda x: x[1])]
    return sorted_folders

def rotate_and_rename_image(src_folders, dest_folders):
    """
    从源文件夹列表中提取图片，顺时针旋转90度，重命名，并保存到目标文件夹列表中。

    Args:
        src_folders (list): 源文件夹路径列表。
        dest_folders (list): 目标文件夹路径列表。
    """
    for src_folder, dest_folder in zip(src_folders, dest_folders):
        src_path = os.path.join(src_folder, "PRED_MAP_plot.png")
        dest_path = os.path.join(dest_folder, "segmentation_PRED_MAP_plot.png")
        
        # 检查源图片是否存在
        if os.path.exists(src_path):
            # 打开图片
            with Image.open(src_path) as img:
                # 顺时针旋转90度
                rotated_img = img.rotate(-90, expand=True)
                # 保存到目标路径
                rotated_img.save(dest_path)
                print(f"Image saved to {dest_path}")
        else:
            print(f"Image not found in {src_path}")

def image_integrater(MATERIAL_PATH, INTEGRATE_PATH):
    # 遍历MATERIAL_PATH下的每个子文件夹
    for subdir in os.listdir(MATERIAL_PATH):
        subdir_path = os.path.join(MATERIAL_PATH, subdir)
        if os.path.isdir(subdir_path):
            cam_images = []
            segmentation_images = []
            # 遍历子文件夹中的文件
            for file in sorted(os.listdir(subdir_path)):
                img_path = os.path.join(subdir_path, file)
                if file.endswith('.jpg'):
                    if 'pvimg_before_norm' in file:
                        cam_images.append((Image.open(img_path), file))
                    elif 'segmentation' in file:  # 匹配所有包含'segmentation'的图像文件
                        if file == "segmentation_PRED_MAP_plot.png":  # 特殊处理这个文件
                            with Image.open(img_path) as img:
                                # 计算新的宽度以保持宽高比
                                new_height = 200
                                aspect_ratio = img.width / img.height
                                new_width = int(new_height * aspect_ratio)
                                img_resized = img.resize((new_width, new_height), Image.ANTIALIAS)
                                segmentation_images.append(img_resized)
                        else:  # 其他'segmentation'图像正常添加
                            segmentation_images.append(Image.open(img_path))

            # 此处应当开始创建一个新图像，背景为白色
            new_image = Image.new('RGB', (1920, 1080), 'white')

            # 计算segmentation_images扩大后的高度
            seg_height = segmentation_images[0].height * 2 if segmentation_images else 0
            # 假设cam_images高度相同，计算两行的总高度
            cam_height = 2 * (cam_images[0][0].height if cam_images else 0)
            # 计算间隔（简单示例，可根据需要调整）
            gap = 50 if cam_images and segmentation_images else 0
            # 计算总高度
            total_height = seg_height + cam_height + gap
            
            # 确定起始y坐标以确保整体垂直居中
            start_y = (1080 - total_height) // 2
            
            # 粘贴segmentation_images
            # 计算segmentation_images扩大后的总宽度，包括间隙
            total_seg_width = sum(img.width * 2 for img in segmentation_images) + (len(segmentation_images) - 1) * 10

            # 确定起始x坐标以确保整体居中，包括间隙
            start_x_seg = (1920 - total_seg_width) // 2
            x_offset = start_x_seg
            for img in segmentation_images:
                seg_img_resized = img.resize((img.width * 2, img.height * 2))
                new_image.paste(seg_img_resized, (x_offset, start_y))
                x_offset += seg_img_resized.width + 10
            
            # 粘贴cam_images
            start_x_cam = (1920 - 3 * 496) // 2  # 假设每行最多3张图
            y_offset = start_y + seg_height + gap  # 更新y坐标偏移量为segmentation_images的高度加上间隔
            for img, name in cam_images:
                first_char = name[0]
                row_offset = 0 if first_char in ['1', '2', '3'] else cam_images[0][0].height
                col = int(first_char) - 1 if first_char in ['1', '2', '3'] else abs(int(first_char) - 6) # int(first_char) - 4 or abs(int(first_char) - 6)
                new_image.paste(img, (start_x_cam + col * 496, y_offset + row_offset))
            
            # 保存合成图
            seg_img_name = segmentation_images[0].filename.split('/')[-1] if segmentation_images else 'default'
            output_filename = '_'.join(seg_img_name.split('_')[:-1]) + '.jpg'
            output_path = os.path.join(INTEGRATE_PATH, output_filename)
            new_image.save(output_path, format='JPEG')
            print(f"Saved integrated image to {output_path}")

def img_extracter_with_MapTR(MATERIAL_PATH, MAPTR_OUTPUT_PATH):
    subfolders = list(get_subfolders(MAPTR_OUTPUT_PATH))
    # 使用sorted()函数和自定义排序键来排序
    sorted_subfolders = sorted(subfolders, key=lambda x: (not 'n015' in x, x)) # MapTR outputs

    sorted_folders = sort_folders_by_min_image_number(MATERIAL_PATH) # HMN outputs

    rotate_and_rename_image(sorted_subfolders, sorted_folders)
    #print(sorted_subfolders)
    # for folder in sorted_folders:
    #     print(folder)
    

if __name__ == '__main__':
    # img_extracter_with_MapTR(MATERIAL_PATH, MAPTR_OUTPUT_PATH)
    image_integrater(MATERIAL_PATH, INTEGRATE_PATH)
    pass

Saved integrated image to ./integrate_images/segmentation_000032_kLcf5dyqVBQZ2OrPdecBgy5Z49PptQTq_000.jpg
Saved integrated image to ./integrate_images/segmentation_000036_F6leDVdKDv0at4tFmuMELVwobB9T99gG_000.jpg
Saved integrated image to ./integrate_images/segmentation_000026_7kcxdkAXQbriTPWWzZlEactu6rlcGIxX_000.jpg
Saved integrated image to ./integrate_images/segmentation_000030_o5f6trtJm7HSTBMTJepJ4u1W7fUIlxVE_000.jpg
Saved integrated image to ./integrate_images/segmentation_000031_jyGG9MMYqfRoU2wGKoJuNnSn11gTeOW1_000.jpg
Saved integrated image to ./integrate_images/segmentation_000038_Ol1zuL3Qxa6qR0HNhdczcussOeXI4jQM_000.jpg
Saved integrated image to ./integrate_images/segmentation_000012_19MBkOV1BAMtH28loWOpOehXU0PneLrL_000.jpg
Saved integrated image to ./integrate_images/segmentation_000018_19uyTNWYMa4aPuoZL0O1sog1L41ZKmIk_000.jpg
Saved integrated image to ./integrate_images/segmentation_000024_QhGsuRgNEHcSTC0aZTvDFv1SSkVHpqkG_000.jpg
Saved integrated image to ./integrate_images/s